In [ ]:
from ultralytics import YOLO
import cv2
import time

# Paths to the YOLOv8 model weights
models = {
    'YOLOv8n': 'yolov8n.pt',  # nano - fastest, smallest
    'YOLOv8s': 'yolov8s.pt',  # small
    'YOLOv8m': 'yolov8m.pt'   # medium - more accurate, slower
}

video_path = '/home/sayeed/no_face_video.mp4'

# Iterate over models to evaluate each
for name, weight in models.items():
    print(f"\n🔍 Running {name}...")

    # Load model
    model = YOLO(weight)

    # Open video
    cap = cv2.VideoCapture(video_path)
    width, height = int(cap.get(3)), int(cap.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'{name}_output.mp4', fourcc, 30, (width, height))

    frame_count = 0
    total_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start = time.time()
        results = model(frame)  # Run YOLOv8 detection
        end = time.time()

        # Timing
        inference_time = end - start
        total_time += inference_time
        frame_count += 1

        # Annotated frame
        annotated_frame = results[0].plot()
        out.write(annotated_frame)

        # Optional: show in real time
        cv2.imshow(name, annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    avg_fps = frame_count / total_time
    print(f"✅ {name} done. Processed {frame_count} frames.")
    print(f"⏱️ Average FPS: {avg_fps:.2f}")
